In [1]:
# Install Dependencies
# This specific package of tf is needed to agree with keras-rl2.
!pip install tensorflow==2.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing 

In [2]:
!pip install keras-rl2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.0 MB/s eta 0:00:00


In [3]:
# Import Dependencies
import random
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Different agents within the Keras model
from rl.agents.dqn import DQNAgent
# policy based RL. Specific policy
from rl.policy import BoltzmannQPolicy
# For our agent we need to maintain memory
from rl.memory import SequentialMemory

Setting up RPSLS Environment

In [4]:
# Set up environment for agent to learn. Includes defining actions and rewards.
class RockPaperScissorsLizardSpockEnvironment:
     def __init__(self):
      self.actions = [0, 1, 2, 3, 4]  # rock: 0, paper: 1, scissors: 2, lizard: 3, spock: 4
      self.rewards = {(0, 0): 0, (0, 1): -1, (0, 2): 1, (0, 3): 1, (0, 4): -1,
       (1, 0): 1, (1, 1): 0, (1, 2): -1, (1, 3): -1, (1, 4): 1, (2, 0): -1,
        (2, 1): 1, (2, 2): 0, (2, 3): 1, (2, 4): -1, (3, 0): -1, (3, 1): 1,
         (3, 2): -1, (3, 3): 0, (3, 4): 1, (4, 0): 1, (4, 1): -1, (4, 2): 1, (4, 3): -1, (4, 4): 0
                      }

      self.max_steps = 10
      self.current_step = 0


     def step(self, action):
        #encoded_action = encode_action(action)
        opponent_action = random.choice(self.actions)
        reward = self.rewards[(action, opponent_action)]
        self.current_step += 1
        done = self.current_step >= self.max_steps
        #encoded_state = encode_state(opponent_action)
        info = {}
        return opponent_action, reward, done, info

     def reset(self):
        self.current_step = 0
        action = random.choice(self.actions)
        #encoded_action = encode_action(action)
        return action


In [5]:
# Define the Q-network model 2
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=(1,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
# Set up the environment and agent
env = RockPaperScissorsLizardSpockEnvironment()
states = 1
actions = [0, 1, 2, 3, 4]
model = build_model(states, len(actions))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [7]:
# Summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 5)                 125       
                                                                 
Total params: 773
Trainable params: 773
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Set up DQN agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, memory=memory, policy=policy,
               nb_actions=len(actions), nb_steps_warmup=10, target_model_update=1e-2)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [9]:
# Train the DQN agent
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 8:47 - reward: 1.0000

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.10/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 74s 7ms/step - reward: 0.0203
1000 episodes - episode_reward: 0.203 [-8.000, 8.000] - loss: 0.408 - mae: 0.515 - mean_q: 0.504

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 0.0067
1000 episodes - episode_reward: 0.067 [-8.000, 9.000] - loss: 0.408 - mae: 0.510 - mean_q: 0.475

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 76s 8ms/step - reward: 0.0017
1000 episodes - episode_reward: 0.017 [-7.000, 10.000] - loss: 0.405 - mae: 0.418 - mean_q: 0.350

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 0.0077
1000 episodes - episode_reward: 0.077 [-9.000, 10.000] - loss: 0.403 - mae: 0.363 - mean_q: 0.275

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 82s 8ms/step - reward: -0.0028
done, took 392.735 seconds


In [10]:
# Evaluate the trained agent
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 1.000, steps: 10
Episode 2: reward: 1.000, steps: 10
Episode 3: reward: -2.000, steps: 10
Episode 4: reward: 5.000, steps: 10
Episode 5: reward: -5.000, steps: 10
Episode 6: reward: -2.000, steps: 10
Episode 7: reward: 3.000, steps: 10
Episode 8: reward: -4.000, steps: 10
Episode 9: reward: 0.000, steps: 10
Episode 10: reward: -1.000, steps: 10
Episode 11: reward: -6.000, steps: 10
Episode 12: reward: -1.000, steps: 10
Episode 13: reward: 3.000, steps: 10
Episode 14: reward: 3.000, steps: 10
Episode 15: reward: 0.000, steps: 10
Episode 16: reward: -1.000, steps: 10
Episode 17: reward: 3.000, steps: 10
Episode 18: reward: -3.000, steps: 10
Episode 19: reward: -2.000, steps: 10
Episode 20: reward: 0.000, steps: 10
Episode 21: reward: 0.000, steps: 10
Episode 22: reward: -1.000, steps: 10
Episode 23: reward: 1.000, steps: 10
Episode 24: reward: -7.000, steps: 10
Episode 25: reward: 2.000, steps: 10
Episode 26: reward: 7.000, steps: 10
Episod

In [11]:
# Calculate sum, mean and median of the rewards
sum_reward = np.sum(scores.history['episode_reward'])
mean_reward = np.mean(scores.history['episode_reward'])
median_reward = np.median(scores.history['episode_reward'])

print("Sum Reward:", sum_reward)
print("Mean Reward:", mean_reward)
print("Median Reward:", median_reward)

Sum Reward: 1.0
Mean Reward: 0.01
Median Reward: 0.0
